<a href="https://colab.research.google.com/github/Deep-Dey1/LLM-models/blob/main/CustomLLM_Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**(Fine Tuning) LLM model for Custom use**

---



**Installing required libraries**

In [4]:
!pip install transformers datasets accelerate peft bitsandbytes trl wandb


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.9/313.9 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 22.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [1]:
!pip uninstall -y bitsandbytes
!pip install --upgrade bitsandbytes
!pip install --upgrade transformers accelerate


Found existing installation: bitsandbytes 0.45.1
Uninstalling bitsandbytes-0.45.1:
  Successfully uninstalled bitsandbytes-0.45.1
  Using cached bitsandbytes-0.45.1-py3-none-manylinux_2_24_x86_64.whl.metadata (5.8 kB)
Using cached bitsandbytes-0.45.1-py3-none-manylinux_2_24_x86_64.whl (69.7 MB)


HUGGING FACE AUTHENTICATION FOR MODEL USE

In [1]:
from huggingface_hub import login

login()


In [2]:
login("your hugging face api")


Load Mistral 7B Model

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

model_name = "mistralai/Mistral-7B-v0.1"

# Define 4-bit quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",  # Normal Float 4, better for efficiency
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True  # Helps with lower VRAM
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load model with updated quantization method
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,  # Use BitsAndBytesConfig
    device_map="auto"  # Assigns model to GPU automatically
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Load Academic Dataset (SQuAD)

In [5]:
from datasets import load_dataset

dataset = load_dataset("squad_v2")

# Preview dataset
print(dataset["train"][0])


README.md:   0%|          | 0.00/8.92k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/16.4M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.35M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

{'id': '56be85543aeaaa14008c9063', 'title': 'Beyoncé', 'context': 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".', 'question': 'When did Beyonce start becoming popular?', 'answers': {'text': ['in the late 1990s'], 'answer_start': [269]}}


DATASET STRUCTURE

In [7]:
print(dataset)


DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 130319
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11873
    })
})


Apply Tokenization to the Dataset

In [14]:
def preprocess_function(examples):
    inputs = [q.strip() + " " + c.strip() for q, c in zip(examples["question"], examples["context"])]
    targets = [a["text"][0].strip() if len(a["text"]) > 0 else "" for a in examples["answers"]]

    # Tokenize inputs
    model_inputs = tokenizer(
        inputs,
        max_length=512,
        truncation=True,
        padding="max_length"
    )

    # Tokenize targets (labels)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            max_length=512,  # Ensure labels are the same length as input
            truncation=True,
            padding="max_length"
        )

    model_inputs["labels"] = labels["input_ids"]

    # Replace padding token (-100 is used to ignore padding loss)
    model_inputs["labels"] = [
        [(l if l != tokenizer.pad_token_id else -100) for l in label]
        for label in model_inputs["labels"]
    ]

    return model_inputs

# Apply preprocessing to dataset
tokenized_dataset = dataset.map(preprocess_function, batched=True)

# Remove unused columns
tokenized_dataset = tokenized_dataset.remove_columns(["id", "title", "context", "question", "answers"])
tokenized_dataset.set_format("torch")  # Convert to PyTorch format


Map:   0%|          | 0/130319 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3961: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/11873 [00:00<?, ? examples/s]

In [16]:
small_dataset = tokenized_dataset["train"].shuffle(seed=42).select(range(5000))  # Use first 5k samples


In [17]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_dataset,  # Use the small dataset for training
    eval_dataset=tokenized_dataset["validation"],  # Use full validation set for evaluation
)


In [ ]:
trainer.train()


Epoch,Training Loss,Validation Loss


Full SCALE TRAINING

In [15]:
from transformers import TrainingArguments, Trainer

# Define training arguments
training_args = TrainingArguments(
    output_dir="./mistral_finetuned",
    per_device_train_batch_size=1,  # Adjust based on GPU memory
    gradient_accumulation_steps=4,  # Helps with low VRAM usage
    learning_rate=2e-4,
    num_train_epochs=3,
    save_steps=1000,
    evaluation_strategy="epoch",
    save_total_limit=2,
    fp16=True,  # Use mixed precision for efficiency
    logging_dir="./logs",
    report_to="wandb",  # Enable Weights & Biases logging
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],  # Use the preprocessed dataset
    eval_dataset=tokenized_dataset["validation"],  # Use validation set for evaluation
)

# Start training
trainer.train()


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [12]:
trainer.train()


ValueError: Expected input batch_size (511) to match target batch_size (127).